# SETUP PART

In [19]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
import numpy as np
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.11.0


# GET MODEL INPUT AND LABELS

In [20]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

### Input Data Quantization

In [12]:
INPUT_S = 0.003921568859368563
INPUT_Z = 128

NUMBER_OF_TEST = 10000

Make C array for input data

In [13]:
INPUT_SIZE_STR  = "const int model_input_dims[] = {"\
                + str(test_images[0].shape[0]) + ", "\
                + str(test_images[0].shape[1]) + ", "\
                +  "1};\n\n"

NUMBER_OF_TEST_STR = "const int number_of_test = " + str(NUMBER_OF_TEST) + ";\n"

TEST_IMAGE_ARR  = "const int8_t test_images[]["\
                + str(test_images[0].shape[0]) + " * "\
                + str(test_images[0].shape[1]) + " * "\
                + "1] = {\n"

for idx in range(NUMBER_OF_TEST):
    TEST_IMAGE_ARR += "\t{\n"
    for line in test_images[idx]:
        TEST_IMAGE_ARR += '\t\t'
        for ele in line:
            ele_quant = int((ele/INPUT_S) - 128)
            TEST_IMAGE_ARR += str(ele_quant) + ', '
        TEST_IMAGE_ARR += '\n'
    TEST_IMAGE_ARR += "\t},\n"

TEST_IMAGE_ARR = TEST_IMAGE_ARR[:-2] + "\n};"

TEST_LABEL_ARR = "const int8_t test_labels[] = {\n\t"
for idx in range(NUMBER_OF_TEST):
    TEST_LABEL_ARR += str(test_labels[idx]) + ', '
    
TEST_LABEL_ARR = TEST_LABEL_ARR[:-2] + "\n};\n"

with open("out_log/model_input.log", "w") as log:
    log.write("#include \"mnist_input_data.h\"\n\n")
    log.write(INPUT_SIZE_STR)
    log.write(NUMBER_OF_TEST_STR)
    log.write(TEST_LABEL_ARR)
    log.write(TEST_IMAGE_ARR)

### Custom for int32_t* data

In [5]:
NUMBER_OF_TEST_STR = "const int number_of_test = " + str(NUMBER_OF_TEST) + ";\n"

TEST_IMAGE_ARR  = "const int8_t test_images[]["\
                + str(test_images[0].shape[0]) + " * "\
                + str(test_images[0].shape[1]) + " * "\
                + "1] = {\n"

for idx in range(NUMBER_OF_TEST):
    TEST_IMAGE_ARR += "\t{\n"
    for line in test_images[idx]:
        TEST_IMAGE_ARR += '\t\t'
        for ele in line:
            ele_quant = int((ele/INPUT_S) - 128)
            TEST_IMAGE_ARR += str(ele_quant) + ', '
        TEST_IMAGE_ARR += '\n'
    TEST_IMAGE_ARR += "\t},\n"

TEST_IMAGE_ARR = TEST_IMAGE_ARR[:-2] + "\n};"

TEST_LABEL_ARR = "const int8_t test_labels[] = {\n\t"
for idx in range(NUMBER_OF_TEST):
    TEST_LABEL_ARR += str(test_labels[idx]) + ', '
    
TEST_LABEL_ARR = TEST_LABEL_ARR[:-2] + "\n};\n"

with open("out_log/test_model_input.log", "w") as log:
    log.write("#include \"mnist_data.h\"\n\n")
    log.write(NUMBER_OF_TEST_STR)
    log.write(TEST_IMAGE_ARR)

# GET MODEL PARAMETERS

In [21]:
MODEL_PARAM_DIR = "./cnnnew_mnist_model_parameters/"

## First CONV Layer

Kernel C Array

In [22]:
cnn_conv0_kernel = np.load(MODEL_PARAM_DIR + "cnnnew_conv0_kernel_tensor.npy")

CNN_CONV0_KERNEL_size_str = "const int cnnnew_conv0_kernel_dims[] = {" + \
                            str(cnn_conv0_kernel.shape[1]) + ", " + \
                            str(cnn_conv0_kernel.shape[2]) + ", " + \
                            str(cnn_conv0_kernel.shape[3]) + ", " + \
                            str(cnn_conv0_kernel.shape[0]) + "};\n\n" 

CNN_CONV0_KERNEL_arr_str = "const int8_t cnnnew_conv0_kernel_data[] = {\n"

for nok_i in range(cnn_conv0_kernel.shape[0]):
    for c_i in range(cnn_conv0_kernel.shape[3]):
        CNN_CONV0_KERNEL_arr_str += '\t'
        for h_i in range(cnn_conv0_kernel.shape[2]):
            for w_i in range(cnn_conv0_kernel.shape[1]):
                CNN_CONV0_KERNEL_arr_str += str(cnn_conv0_kernel[nok_i][h_i][w_i][c_i]) + ', '
        CNN_CONV0_KERNEL_arr_str += '\n'

CNN_CONV0_KERNEL_arr_str = CNN_CONV0_KERNEL_arr_str[:-3] + "\n};"

In [23]:
with open("debug_log/cnnnew_conv0_kernel.log", "w") as log:
    log.write(CNN_CONV0_KERNEL_size_str)
    log.write(CNN_CONV0_KERNEL_arr_str)

Bias C Array

In [24]:
cnn_conv0_bias = np.load(MODEL_PARAM_DIR + "cnnnew_conv0_bias_tensor.npy")

CNN_CONV0_BIAS_size_str = "const int cnnnew_conv0_bias_dims[] = {" + str(cnn_conv0_bias.shape[0]) +"};\n\n"

CNN_CONV0_BIAS_arr_str = "const int32_t cnnnew_conv0_bias_data[] = {\n"

CNN_CONV0_BIAS_arr_str += '\t'
for nok_i in range(cnn_conv0_bias.shape[0]):
    CNN_CONV0_BIAS_arr_str += str(cnn_conv0_bias[nok_i]) + ', '
    
CNN_CONV0_BIAS_arr_str = CNN_CONV0_BIAS_arr_str[:-2] + "\n};"

In [25]:
with open("debug_log/cnnnew_conv0_bias.log", "w") as log:
    log.write(CNN_CONV0_BIAS_size_str)
    log.write(CNN_CONV0_BIAS_arr_str)

## Second CONV Layer

Kernel C Array

In [26]:
cnn_conv1_kernel = np.load(MODEL_PARAM_DIR + "cnnnew_conv1_kernel_tensor.npy")

CNN_CONV1_KERNEL_size_str = "const int cnnnew_conv1_kernel_dims[] = {" + \
                            str(cnn_conv1_kernel.shape[1]) + ", " + \
                            str(cnn_conv1_kernel.shape[2]) + ", " + \
                            str(cnn_conv1_kernel.shape[3]) + ", " + \
                            str(cnn_conv1_kernel.shape[0]) + "};\n\n" 

CNN_CONV1_KERNEL_arr_str = "const int8_t cnnnew_conv1_kernel_data[] = {\n"

for nok_i in range(cnn_conv1_kernel.shape[0]):
    for c_i in range(cnn_conv1_kernel.shape[3]):
        CNN_CONV1_KERNEL_arr_str += '\t'
        for h_i in range(cnn_conv1_kernel.shape[2]):
            for w_i in range(cnn_conv1_kernel.shape[1]):
                CNN_CONV1_KERNEL_arr_str += str(cnn_conv1_kernel[nok_i][h_i][w_i][c_i]) + ', '
        CNN_CONV1_KERNEL_arr_str += '\n'

CNN_CONV1_KERNEL_arr_str = CNN_CONV1_KERNEL_arr_str[:-3] + "\n};"

In [27]:
with open("debug_log/cnnnew_conv1_kernel.log", "w") as log:
    log.write(CNN_CONV1_KERNEL_size_str)
    log.write(CNN_CONV1_KERNEL_arr_str)

Bias C Array

In [28]:
cnn_conv1_bias = np.load(MODEL_PARAM_DIR + "cnnnew_conv1_bias_tensor.npy")

CNN_CONV1_BIAS_size_str = "const int cnnnew_conv1_bias_dims[] = {" + str(cnn_conv1_bias.shape[0]) +"};\n\n"

CNN_CONV1_BIAS_arr_str = "const int32_t cnnnew_conv1_bias_data[] = {\n"

CNN_CONV1_BIAS_arr_str += '\t'
for nok_i in range(cnn_conv1_bias.shape[0]):
    CNN_CONV1_BIAS_arr_str += str(cnn_conv1_bias[nok_i]) + ', '
    
CNN_CONV1_BIAS_arr_str = CNN_CONV1_BIAS_arr_str[:-2] + "\n};"

In [29]:
with open("debug_log/cnnnew_conv1_bias.log", "w") as log:
    log.write(CNN_CONV1_BIAS_size_str)
    log.write(CNN_CONV1_BIAS_arr_str)

## First DENSE Layer

Weight C Array (contain reshape)

In [30]:
#     for conv_c_i in range(12):
#         for conv_h_i in range(13):
#             for conv_w_i in range(13):
#                 LENET_DENSE0_WEIGHT_arr_str += str(lenet_dense0_weight[dense_h_i][156*conv_h_i + 12*conv_w_i + conv_c_i]) + ', '


In [31]:
cnn_conv1_shape = [5, 5, 32]
# NEED CHANGE HERE!!!!

cnn_dense0_weight = np.load(MODEL_PARAM_DIR + "cnnnew_dense0_weight_tensor.npy")

CNN_DENSE0_WEIGHT_size_str = "const int cnnnew_dense0_weight_dims[] = {" + \
                            str(cnn_dense0_weight.shape[0]) + ", " + \
                            str(cnn_dense0_weight.shape[1]) + "};\n\n"

CNN_DENSE0_WEIGHT_arr_str = "const int8_t cnnnew_dense0_weight_data[] = {\n"

for dense_h_i in range(cnn_dense0_weight.shape[0]):
    CNN_DENSE0_WEIGHT_arr_str += '\t'
    for conv_c_i in range(cnn_conv1_shape[2]):
        for conv_h_i in range(cnn_conv1_shape[1]):
            for conv_w_i in range(cnn_conv1_shape[0]):
                CNN_DENSE0_WEIGHT_arr_str += str(cnn_dense0_weight[dense_h_i][\
                    cnn_conv1_shape[2]*cnn_conv1_shape[1]*conv_h_i + \
                    cnn_conv1_shape[2]*conv_w_i + conv_c_i]) + ', '
    CNN_DENSE0_WEIGHT_arr_str += '\n'

CNN_DENSE0_WEIGHT_arr_str = CNN_DENSE0_WEIGHT_arr_str[:-3] + "\n};"

In [32]:
with open("debug_log/cnnnew_dense0_weight.log", "w") as log:
    log.write(CNN_DENSE0_WEIGHT_size_str)
    log.write(CNN_DENSE0_WEIGHT_arr_str)

Bias C Array

In [33]:
cnn_dense0_bias = np.load(MODEL_PARAM_DIR + "cnnnew_dense0_bias_tensor.npy")

CNN_DENSE0_BIAS_size_str = "const int cnnnew_dense0_bias_dims[] = {" + str(cnn_dense0_bias.shape[0]) + "};\n\n"

CNN_DENSE0_BIAS_arr_str = "const int32_t cnnnew_dense0_bias_data[] = {\n"

CNN_DENSE0_BIAS_arr_str += '\t'
for nok_i in range(cnn_dense0_bias.shape[0]):
    CNN_DENSE0_BIAS_arr_str += str(cnn_dense0_bias[nok_i]) + ', '
    
CNN_DENSE0_BIAS_arr_str = CNN_DENSE0_BIAS_arr_str[:-2] + "\n};"

In [34]:
with open("debug_log/cnnnew_dense0_bias.log", "w") as log:
    log.write(CNN_DENSE0_BIAS_size_str)
    log.write(CNN_DENSE0_BIAS_arr_str)